# Retrieving Data with Propublica API

In [11]:
import numpy as np
import pandas as pd
import pickle

import requests
import json
import configparser

from bs4 import BeautifulSoup
import urllib.request
import re

In [ ]:
# Config API
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config.get('propublica', 'PROPUBLICA_API_KEY')

## Retrieve Senators
Through ProPublica, a list of senators can be retrieved starting from the 101st Congress (1989-1990) to the 116th Congress (2019-2021).

In [33]:
# Set parameters
congresses = range(101, 117) # from the 101tst congress to the 116th congress

In [ ]:
# Get all congresses
list_of_congresses = []

for n in congresses:
    r = requests.get(
        f'https://api.propublica.org/congress/v1/{n}/senate/members.json',
        headers={'X-API-Key': api_key}
    )
    
    results = r.json()['results'][0]
    members = results['members']
    list_of_congresses.append(members)

In [ ]:
# Function to grab data for senators
def get_senators(members):
    senators = []
    for member in members:
        senator = {
            'id': member['id'],
            'first_name': member['first_name'],
            'last_name': member['last_name'],
            'party': member['party'],
            'gender': member['gender'],
            'state': member['state'],
        }
        senators.append(senator)
    return senators

In [ ]:
senates = []
for members in list_of_congresses:
    senators = get_senators(members)
    senates.append(senators)

In [ ]:
# Create list of all unique senators across 101st to 116th congress
senator_ids = []
all_senators = []
for senate in senates:
    for senator in senate:
        if senator['id'] in senator_ids:
            continue
        else:
            senator_ids.append(senator['id'])
            all_senators.append(senator)

In [ ]:
print('Senators:', len(all_senators))

In [ ]:
# with open('temp_senators.p', 'wb') as f:
#     pickle.dump(all_senators, f)

## Web Scrape Total Roll Call Votes
The API only returns the 20 most recent votes by a senator so searching votes by senator is not possible.  Instead, votes will be retrieve through votes by roll call.  Each senate session (2 sessions per congress) has a varying number of bills voted on so the total number of roll call votes needs to be web scraped.  This is done on the [US Senate](https://www.senate.gov/) website.

In [2]:
# Create BeautifulSoup object to find number of roll call votes for each session
html = urllib.request.urlopen('https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_101_1.htm')
soup = BeautifulSoup(html, 'lxml')

In [13]:
# Element holding last roll call vote in session
last = soup.find('td').find('a').text
last

'312\xa0(99-0)'

In [32]:
# Last roll call vote in session
re.search(r'^...(?=\\)*', last).group(0)

'312'

In [34]:
# Script to pull total roll call votes per session
roll_calls = {}
for congress in congresses:
    # First session
    html = urllib.request.urlopen(
        f'https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_{congress}_1.htm'
    )
    soup = BeautifulSoup(html, 'lxml')
    last = soup.find('td').find('a').text
    last_roll_call = re.search(r'^...(?=\\)*', last).group(0)
    roll_calls[f'{congress}_1'] = int(last_roll_call)
    
    # Second session
    html = urllib.request.urlopen(
        f'https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_{congress}_2.htm'
    )
    soup = BeautifulSoup(html, 'lxml')
    last = soup.find('td').find('a').text
    last_roll_call = re.search(r'^...(?=\\)*', last).group(0)
    roll_calls[f'{congress}_2'] = int(last_roll_call)

In [41]:
sum(roll_calls.values())

10234

## Retrieve Votes by Roll Calls
There is a limit of 5000 API calls per day for ProPublica.  So in order to be safe, the roll calls will be split into chunks of 2000, 4000, and 4234.